In [1]:
import pymongo

In [2]:
#connect to database
myclient = pymongo.MongoClient("mongodb://localhost:27017")

print(myclient)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [3]:
mydb = myclient['python_dev']

In [4]:
# return a list of databases
print(myclient.list_database_names())

['Catalog', 'ProductDB', 'admin', 'config', 'local']
